<a href="https://colab.research.google.com/github/saurabhshinkar/CRP/blob/main/Cardiovascular_Risk_prediction_Saurabh_Shinkar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Title : Cardiovascular Risk Prediction**

## <b> Problem Description </b>

###The dataset is from an ongoing cardiovascular study on residents of the town of Framingham,Massachusetts. The classification goal is to predict whether the patient has a 10-year risk of future coronary heart disease (CHD). 

## <b> Data Description </b>

### <b> The dataset provides the patients’ information. It includesover 4,000 records and 15 attributes.Variables Each attribute is a potential risk factor. There are both demographic, behavioral, and medical risk factors.</b>

### <b>Demographic: </b>


• **Sex**: male or female("M" or "F")

•** Age**: Age of the patient;(Continuous - Although the recorded ages have been truncated to
whole numbers, the concept of age is continuous)
Behavioral

• **is_smoking**: whether or not the patient is a current smoker ("YES" or "NO")

• **Cigs Per Day**: the number of cigarettes that the person smoked on average in one day.(can be
considered continuous as one can have any number of cigarettes, even half a cigarette.)

### <b>Medical( history): </b>


• **BP Meds**: whether or not the patient was on blood pressure medication (Nominal)

• **Prevalent Stroke**: whether or not the patient had previously had a stroke (Nominal)

• **Prevalent Hyp**: whether or not the patient was hypertensive (Nominal)

• **Diabetes**: whether or not the patient had diabetes (Nominal)

### <b>Medical(current): </b>


• **Tot Chol**: total cholesterol level (Continuous)

• **Sys BP**: systolic blood pressure (Continuous)

• **Dia BP**: diastolic blood pressure (Continuous)

• **BMI**: Body Mass Index (Continuous)

• **Heart Rate**: heart rate (Continuous - In medical research, variables such as heart rate though in
fact discrete, yet are considered continuous because of large number of possible values.)

• **Glucose**: glucose level (Continuous)

### <b>Predict variable (desired target) </b>


• 10-year risk of coronary heart disease CHD(binary: “1”, means “Yes”, “0” means “No”) -
DV

# **Importing** **Libraries**

In [23]:
#Import all library that will be used in entire project

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# **Mount Drive And Import Data**

In [2]:
#Mount google drive for access of the play store dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importing Dataset
File_path='/content/drive/MyDrive/Capstone Projects/Cardiovascular Risk prediction/'
data= pd.read_csv(File_path + 'data_cardiovascular_risk.csv')

In [4]:
# First Look 
data.head()

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,NaN,90.0,80.0,1
1,1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,3,50,1.0,M,YES,20.0,0.0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,4,64,1.0,F,YES,30.0,0.0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0


In [ ]:
#tail of data
data.tail()

In [6]:
#data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3390 entries, 0 to 3389
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3390 non-null   int64  
 1   age              3390 non-null   int64  
 2   education        3303 non-null   float64
 3   sex              3390 non-null   object 
 4   is_smoking       3390 non-null   object 
 5   cigsPerDay       3368 non-null   float64
 6   BPMeds           3346 non-null   float64
 7   prevalentStroke  3390 non-null   int64  
 8   prevalentHyp     3390 non-null   int64  
 9   diabetes         3390 non-null   int64  
 10  totChol          3352 non-null   float64
 11  sysBP            3390 non-null   float64
 12  diaBP            3390 non-null   float64
 13  BMI              3376 non-null   float64
 14  heartRate        3389 non-null   float64
 15  glucose          3086 non-null   float64
 16  TenYearCHD       3390 non-null   int64  
dtypes: float64(9),

In [ ]:
#Discription of Data
data.describe(include='all')

# **Handling Missing Vaules**

In [ ]:
# Missing Value Count Function
def show_missing():
    missing = data.columns[data.isnull().any()].tolist()
    return missing

# Missing data counts and percentage
print('Missing Data Count')
print(data[show_missing()].isnull().sum().sort_values(ascending = False))
print('--'*50)
print('Missing Data Percentage')
print(round(data[show_missing()].isnull().sum().sort_values(ascending = False)/len(data)*100,2))

In [ ]:
#data of null glucose values
data[data['glucose'].isnull()].head()

In [ ]:
#Distribution Plot Of glucose
sns.distplot(data.glucose.dropna())
plt.title('Distribution Plot Of glucose')

In [ ]:
#median of glucose
data.glucose.median()

In [12]:
#Fill NaN Values in glucose Column with Median
data['glucose'] =data['glucose'].fillna(data['glucose'].median())

In [ ]:
#Distribution Plot Of education
sns.distplot(data.education.dropna())
plt.title('Distribution Plot Of education')

In [14]:
#Fill NaN Values in glucose Column with Median
data['education'] =data['education'].fillna(data['education'].median())

In [ ]:
# Missing Value Count Function
def show_missing():
    missing = data.columns[data.isnull().any()].tolist()
    return missing

# Missing data counts and percentage
print('Missing Data Count')
print(data[show_missing()].isnull().sum().sort_values(ascending = False))
print('--'*50)
print('Missing Data Percentage')
print(round(data[show_missing()].isnull().sum().sort_values(ascending = False)/len(data)*100,2))

In [16]:
#Dropping null as they are now negligible 
data.dropna(inplace=True)

In [ ]:
data.info()

In [63]:
#Numerical features
numerical=['age','cigsPerDay','totChol','sysBP','diaBP','BMI','heartRate','glucose']

In [ ]:
#Boxplot and distribution plot of numerical features
for var in numerical:
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    fig = sns.boxplot(y=data[var])
    fig.set_title('')
    fig.set_ylabel(var)
    
    plt.subplot(1, 2, 2)
    fig = sns.distplot(data[var].dropna())
    fig.set_ylabel('')
    fig.set_xlabel(var)

    plt.show()

**We can see outliers are present in our data but We are not them as we have less data**

smote remained to used and also converting features into categorical and combining related feature is remained

# EDA

In [21]:
data.head(2)

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
1,1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0


In [ ]:
#play_store_data.groupby('Genres')['Installs'].sum().reset_index(name="Total Installs").sort_values(by=['Total Installs'], ascending =False).head(5)

## **Univariate Analysis**

***Barplot Of Age Wise Distribution***

In [ ]:
#barplot of age and its counts 
plt.figure(figsize=(12,7))
sns.countplot(x = data['age'])
plt.title('Age Wise Distribution',fontsize=15)
plt.grid()
plt.show()

In [ ]:
#barplot of education and its counts 
plt.figure(figsize=(12,7))
sns.countplot(x = data['education'])
plt.title('Education Wise Distribution',fontsize=15)
plt.grid()
plt.show()

In [ ]:
#Percentage Count Of Male And female
plt.figure(figsize=(10,7))
plt.pie(data['sex'].value_counts(),labels=['M','F'],autopct='%.0f%%',explode=[0.03,0])
plt.title('Percentage Of Male And Female')
plt.show()

In [ ]:
#Categories of Top 50 Install Apps
data['is_smoking'].value_counts().plot(kind="barh")
plt.title("Barh Plot Of is_smoking counts",fontsize=20)
plt.ylabel("is_smoking")
plt.xlabel("Count")
plt.grid()

In [ ]:
#Distribution plot of numerical features
for var in numerical:
    plt.subplot(1, 2, 2)
    fig = sns.distplot(data[var].dropna(),color="brown",fit=norm)
    plt.grid()
    fig.set_ylabel('count')
    fig.set_xlabel(var)

    plt.show()

In [ ]:
#bar plot of numerical features
for var in numerical:
    plt.rcParams['figure.figsize'] = (22,10)
    plt.subplot(1, 2, 2)
    fig = sns.countplot(x=data[var],hue=data['TenYearCHD'])
    plt.xticks(rotation=90, horizontalalignment="center")
    plt.grid()
    fig.set_ylabel('count')
    fig.set_xlabel(var)

    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)
sns.countplot(x = data['sex'],hue = data['TenYearCHD'])